In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

In [6]:
import pandas as pd

data=pd.read_csv("train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [17]:
data=data.dropna()

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot


In [19]:
x=data.drop("label",axis=1)
y=data["label"]

In [20]:
voc_size=5000

In [27]:
messages=x.copy()
messages.reset_index(inplace=True)
len(messages["title"])

18285

In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [31]:
cor=[]
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages["title"][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words("english")]
    review=" ".join(review)
    cor.append(review)
    

In [33]:
one_hot_rep=[one_hot(word,voc_size) for word in cor]

In [42]:
embedded_docs=pad_sequences(one_hot_rep,padding="pre",maxlen=20)

In [43]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   56,
       2832, 4409, 3980, 4549, 1504,  247, 1632, 1773, 3361])

In [46]:
#model

model=Sequential()
model.add(Embedding(voc_size,40,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
import numpy as np

x_final=np.array(embedded_docs)
y_final=np.array(y)

In [52]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [53]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 50ms/step - loss: 0.3414 - accuracy: 0.8419 - val_loss: 0.1914 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1389 - accuracy: 0.9469 - val_loss: 0.1989 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0975 - accuracy: 0.9642 - val_loss: 0.2278 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0684 - accuracy: 0.9756 - val_loss: 0.2493 - val_accuracy: 0.9175
Epoch 5/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0414 - accuracy: 0.9860 - val_loss: 0.3059 - val_accuracy: 0.9056
Epoch 6/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0283 - accuracy: 0.9901 - val_loss: 0.3511 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0184 - accuracy: 0.9942 - val_loss: 0.4031 - val_accuracy: 0.913

In [58]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
pred=model.predict_classes(X_test)
confusion_matrix(y_test,pred)



C:\Users\artig\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3097,  322],
       [ 181, 2435]], dtype=int64)

In [59]:
accuracy_score(y_test,pred)

0.9166528583264292

In [56]:
from tensorflow.keras.layers import Dropout

model=Sequential()
model.add(Embedding(voc_size,40,input_length=20))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 54ms/step - loss: 0.3595 - accuracy: 0.8318 - val_loss: 0.1989 - val_accuracy: 0.9135
Epoch 2/10
192/192 [==============================] - 9s 45ms/step - loss: 0.1640 - accuracy: 0.9340 - val_loss: 0.1992 - val_accuracy: 0.9143
Epoch 3/10
192/192 [==============================] - 8s 41ms/step - loss: 0.1249 - accuracy: 0.9542 - val_loss: 0.1947 - val_accuracy: 0.9195
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.1037 - accuracy: 0.9611 - val_loss: 0.2423 - val_accuracy: 0.9180
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0856 - accuracy: 0.9690 - val_loss: 0.2257 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0716 - accuracy: 0.9736 - val_loss: 0.2763 - val_accuracy: 0.9193
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0632 - accuracy: 0.9757 - val_loss: 0.2995 - val_accuracy: 0.911